In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFace
from sagemaker.huggingface import TrainingCompilerConfig

In [ ]:
s3_url = "s3//......."


# gets role for executing training job
role = sagemaker.get_execution_role()
hyperparameters = {
	'model_name_or_path':'rinna/japanese-gpt2-small',
	'output_dir':'/opt/ml/model',
    #"train_file": "/opt/ml/code/small_train.txt",
    #"validation_file": "/opt/ml/code/test.txt", 
    'num_train_epochs': 10,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 8
}

# git configuration to download our fine-tuning script
#git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
    #source_dir='./examples/pytorch/language-modeling',
    #git_config=git_config,
    entry_point='old_run_clm.py',
	instance_count=1,
    instance_type='ml.p3.2xlarge',
	transformers_version='4.11.0',
	pytorch_version='1.9.0',
    compiler_config = TrainingCompilerConfig(),
    hyperparameters = hyperparameters,
	role=role,
    py_version='py38'
)

# starting the train job
huggingface_estimator.fit({"train": f"{s3_url}/data/small_train.txt", "test": f"{s3_url}/data/test.txt"})

In [ ]:
sagemaker.s3.S3Downloader.download(
    s3_uri=f'{s3_url}/data/small_train.txt',
    local_path="/root/"
)

In [ ]:
sagemaker.s3.S3Downloader.list(
    s3_uri=f'{s3_url}/data/',
)